In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

data = pd.read_csv('data.csv')

In [ ]:
def RSI(dataframe, period):
    '''
    Computes the RSI of a given price series for a given period length
    :param dataframe:
    :param period:
    :return dataframe with rsi:
    '''

    rsi = []

    for stock in dataframe['Symbol'].unique():
        all_prices = dataframe[dataframe['Symbol'] == stock]['Close']
        diff = np.diff(all_prices) # length is 1 less than the all_prices
        for i in range(period):
            rsi.append(None) # because RSI can't be calculated until period prices have occured

        for i in range(len(diff) - period + 1):
            avg_gain = diff[i:period + i]
            avg_loss = diff[i:period + i]
            avg_gain = abs(sum(avg_gain[avg_gain >= 0]) / period)
            avg_loss = abs(sum(avg_loss[avg_loss < 0]) / period)
            if avg_loss == 0:
                rsi.append(100)
            elif avg_gain == 0:
                rsi.append(0)
            else:
                rs = avg_gain / avg_loss
                rsi.append(100 - (100 / (1 + rs)))

    dataframe['RSI'] = rsi
    return dataframe


def PROC(dataframe, period):
    '''
    Computes the PROC(price rate of change) of a given price series for a given period length
    :param dataframe:
    :param period:
    :return proc:
    '''

    proc = []

    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        for i in range(period):
            proc.append(None) # because proc can't be calculated until period prices have occured
        for i in range(len(all_prices) - period):
            if len(all_prices) <= period:
                proc.append(None)
            else:
                calculated = (all_prices[i + period] - all_prices[i]) / all_prices[i]
                proc.append(calculated)
    dataframe['PROC'] = proc
    return dataframe


def SO(dataframe, period):
    
    so = []
    
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        
        for i in range(period):
            so.append(None)
 
        for i in range(len(all_prices) - period):
            C = all_prices[i]
            H = max(all_prices[i:i+period])
            L = min(all_prices[i:i+period])
            so.append(100 * ((C - L) / (H - L)))

    dataframe['SO'] = so
    return dataframe


def Williams_R(dataframe, period):
    '''
    Williams %R
    Calculates fancy shit for late usage. Nice!

    EXAMPLE USAGE:
    data = pandas.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
    wr = Williams_R(data)
    print(wr)

    '''
    
    wr = []
    
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        for i in range(period):
            wr.append(None) # because proc can't be calculated until period prices have occured
            
        for i in range(period-1,len(all_prices)-1):
            C = all_prices[i]
            H = max(all_prices[i-period+1:i])
            L = min(all_prices[i-period+1:i])
            wr_one = (
                ((H - C) 
                 / (H - L)) * -100
            )
            if wr_one <=-100:
                wr.append(-100)
            elif wr_one >= 100:
                wr.append(100)
            else:
                wr.append(wr_one)
    dataframe["WR"] = wr
    return dataframe


def calculate_targets(df, period):
    
    targets = []

    for stock in df['Symbol'].unique():
        all_prices = list(df[df['Symbol'] == stock]['Close'])
        
        for i in range(0, len(all_prices)-period):
            targets.append(np.sign(all_prices[i+period] - all_prices[i]))
        for i in range(len(all_prices)-period, len(all_prices)):
            targets.append(None)

    df["Target({})".format(period)] = targets
    return df


def On_Balance_Volume(dataframe):
    '''
    Williams %R
    Calculates fancy shit for late usage. Nice!

    EXAMPLE USAGE:
    data = pandas.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
    wr = Williams_R(data)
    print(wr)

    '''
    obv = []
    
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        all_volumes = list(dataframe[dataframe['Symbol'] == stock]['Volume'])
    
        obv.append(dataframe.iloc[0]["Volume"])
        for i in range(1,len(all_prices)):
            C_old = all_prices[i-1]
            C = all_prices[i]
            if(C > C_old):
                obv.append(obv[i-1]+ all_volumes[i])
            elif (C < C_old):
                obv.append(obv[i - 1] - all_volumes[i])
            else:
                obv.append(obv[i-1])

    dataframe['OBV'] = obv
    return dataframe


def delete_bad_data(df):
    for stock in df['Symbol'].unique():
        if df[df["Symbol"]==stock]["Close"].any() < 1:
            df = df.drop(df[df["Symbol"]==stock], axis=0)
        if df[df["Symbol"]==stock]["Volume"].any() == 0:
            df = df.drop(df[df["Symbol"]==stock], axis=0)
        if df[df["Symbol"]==stock]["Open"].any() == None:
            df = df.drop(df[df["Symbol"]==stock], axis=0)
    return df

In [ ]:
data = delete_bad_data(data)
print("Test of deleting bad data:")
try:
    print("Deletion successful:", data[data["Symbol"]=="AAP"]["Close"][5] )
except:
    print("True")

In [ ]:
data = RSI(data,14)
print("RSI: Done")

In [ ]:
data = PROC(data, 14)
print("PROC: Done")

In [ ]:
data = SO(data,14)
print("SO: Done")

In [ ]:
data = Williams_R(data, 14 )
print("Williams_R: Done")

In [ ]:
data["EWMA"] = pd.ewma(data["Close"], com=.5)
print("EWMA: Done")

In [ ]:
data = calculate_targets(data, 1)
data = calculate_targets(data, 3)
data = calculate_targets(data, 5)
data = calculate_targets(data, 10)
data = calculate_targets(data, 14)
data = calculate_targets(data, 30)
print('Targets Done - except 60')

In [ ]:
data

In [ ]:
data.to_csv("./data_preprocessed.csv")

In [ ]:
data_processed = pd.read_csv("data_preprocessed.csv")
print(data_processed[:10])

In [ ]:
stock= data_processed['Symbol'].unique()[0]
plt.figure(figsize=(10,10))

plt.plot(data_processed[data_processed["Symbol"]==stock]["WR"],label="WR")
plt.plot(data_processed[data_processed["Symbol"]==stock]["SO"],label="So")
plt.plot(data_processed[data_processed["Symbol"]==stock]["RSI"],label="RSI")
plt.plot(data_processed[data_processed["Symbol"]==stock]["PROC"],label="PROC")

plt.plot(data_processed[data_processed["Symbol"]==stock]["Close"],label="Close")
plt.legend()